In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import exceptions
from pymongo import MongoClient
from selenium.webdriver.firefox.options import Options

In [2]:
def _parse_element(element, css_selector):
    result = WebDriverWait(element, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))).text
    return result

In [3]:
def parse_email(element):
    item = {}

    item['from_name'] = _parse_element(
        element, 'span[class~="ns-view-message-head-sender-name"]')
    item['from_email'] = _parse_element(
        element, 'span[class~="mail-Message-Sender-Email"]')
    item['date'] = _parse_element(element,
                                  'div[class~="ns-view-message-head-date"]')
    item['subject'] = _parse_element(
        element, 'div[class~="mail-Message-Toolbar-Subject"]')
    item['text_messege'] = _parse_element(
        element, 'div.mail-Message-Body-Content')

    return item

In [4]:
MONGO_URI = 'mongodb://172.17.0.2:27017/'
MONGO_DATABASE = 'mail_db'

client = MongoClient(MONGO_URI)
mongo_base = client[MONGO_DATABASE]
collection = mongo_base['messeges']

Получение драйвера

In [5]:
chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome()
# url = 'https://mail.ru/'
# driver.get(url)

url = 'https://yandex.ru/'
title_site = 'Яндекс'

driver.get(url)

Переход к форме авторизации

In [6]:
assert title_site in driver.title

try:

    mail_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, \
                'desk-notif-card__login-enter-expanded')))

#     mail_button = driver.find_element_by_css_selector(
#         'a[class="button desk-notif-card__login-enter-expanded"]'
#     )
except exceptions.NoSuchElementException:
    print('Mail button not found')

mail_button.click()

driver.switch_to.window(driver.window_handles[-1])

# driver.title


Авторизация

In [7]:

# # print(driver.window_handles)

# driver.current_url
# # if 'Авторизация' in driver.title:
# print(driver.current_url)
# # login_form = driver.find_element_by_class_name('passp-auth-screen')
# # print(login_form)

login_form = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'passp-page-overlay')))


field_login = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, 'login')))
field_login.send_keys('landingorder')
field_login.send_keys(Keys.ENTER)
field_passwd = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.ID, 'passp-field-passwd')))
field_passwd.send_keys('38Popug@ev')
field_passwd.send_keys(Keys.ENTER)



Открываем письмо

In [8]:
first_messege = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located(
        (By.CLASS_NAME, 'mail-MessageSnippet-FromText')
    )
)

first_messege.click()


In [9]:
driver.find_elements_by_class_name('mail-MessageSnippet-FromText')[1].click()





In [10]:
while True:
    try:
        collection.insert_one(parse_email(driver))

        button_next = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'mail-Message-PrevNext_next')))
        button_next.click()
    except exceptions.TimeoutException:
        print('E-mails are over')
        break

driver.quit()

ServerSelectionTimeoutError: 172.17.0.2:27017: [Errno 113] No route to host